In [1]:

import numpy as np 
import pandas as pd 

import os

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import shutil

In [3]:
base_dir = r'C:\Users\Noman\Desktop\Session\GA Based CNN\Data\animal'
working_dir = r'C:\Users\Noman\Desktop\Session\GA Based CNN\Data\working'

train_dir = os.path.join(working_dir, 'train')
val_dir = os.path.join(working_dir, 'val')
test_dir = os.path.join(working_dir, 'test')

for category in ['cat', 'dog']:
    os.makedirs(os.path.join(train_dir, category), exist_ok=True)
    os.makedirs(os.path.join(val_dir, category), exist_ok=True)
    os.makedirs(os.path.join(test_dir, category), exist_ok=True)

def split_data(SOURCE, TRAINING, VALIDATION, TESTING, split_size):
    files = [f for f in os.listdir(SOURCE) if os.path.isfile(os.path.join(SOURCE, f))]
    files.sort()
    
    train_files, temp_files = train_test_split(files, test_size=(1-split_size[0]), random_state=42)
    val_files, test_files = train_test_split(temp_files, test_size=(split_size[2]/(split_size[1]+split_size[2])), random_state=42)

    for file_name in train_files:
        shutil.copy(os.path.join(SOURCE, file_name), os.path.join(TRAINING, file_name))

    for file_name in val_files:
        shutil.copy(os.path.join(SOURCE, file_name), os.path.join(VALIDATION, file_name))

    for file_name in test_files:
        shutil.copy(os.path.join(SOURCE, file_name), os.path.join(TESTING, file_name))

split_size = (0.7, 0.15, 0.15)

split_data(os.path.join(base_dir, 'cat'), os.path.join(train_dir, 'cat'), os.path.join(val_dir, 'cat'), os.path.join(test_dir, 'cat'), split_size)
split_data(os.path.join(base_dir, 'dog'), os.path.join(train_dir, 'dog'), os.path.join(val_dir, 'dog'), os.path.join(test_dir, 'dog'), split_size)

In [4]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

val_test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(512, 512),
                                                    batch_size=32,
                                                    class_mode='binary')

val_generator = val_test_datagen.flow_from_directory(val_dir,
                                                     target_size=(512, 512),
                                                     batch_size=32,
                                                     class_mode='binary')

test_generator = val_test_datagen.flow_from_directory(test_dir,
                                                      target_size=(512, 512),
                                                      batch_size=32,
                                                      class_mode='binary')

Found 698 images belonging to 2 classes.
Found 150 images belonging to 2 classes.
Found 152 images belonging to 2 classes.


In [5]:
from keras import layers
from keras import models
from keras.callbacks import EarlyStopping

In [6]:
import tensorflow as tf
import keras
from tensorflow.keras import Model
from tensorflow.keras.utils import to_categorical
import keras.backend as K
from keras.models import Sequential, load_model
from keras.layers import Activation, Dropout, Flatten, Dense, LSTM, Conv2D, MaxPooling2D, RepeatVector
from sklearn.metrics import f1_score, ConfusionMatrixDisplay

In [7]:
def build_Classifier(f1, f2, f3, f4, k, a1, a2, a3, a4, d1, op, ep):
    model = Sequential()
    model.add(Conv2D(f1, (k,k), activation = a1, input_shape=(512, 512, 3)))
    model.add(MaxPooling2D((2,2)))
    model.add(Conv2D(f2, (k,k), activation = a2))
    model.add(MaxPooling2D((2,2)))
    model.add(Conv2D(f3, (k,k), activation = a3))
    model.add(Flatten())
    model.add(Dense(f4, activation = a4))
    model.add(Dropout(d1))
    model.add(Dense(1 ,activation = "sigmoid"))

    model.compile(optimizer = op, loss = keras.losses.binary_crossentropy, metric = ["accuracy"])
    es = EarlyStopping(monitor = "val_accuracy", patience = 7)
    model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = ep, batch_size = 100, callbacks = [es], verbose=0)
    return model

In [8]:
from random import choice
from random import uniform
from numpy.random import randint

## Initializing Gene Parameters values

In [9]:
def initialization():
    parameters = {}
    f1 = choice([32, 64])
    parameters["f1"] = f1
    f2 = choice([64, 128])
    parameters["f2"] = f2
    f3 = choice([128, 256])
    parameters["f3"] = f3
    f4 = choice([512])
    parameters["f4"] = f4
    k = choice([3, 5])
    parameters["k"] = k
    a1 = choice(["relu"])
    parameters["a1"] = a1
    a2 = choice(["relu"])
    parameters["a2"] = a2
    a3 = choice(["relu"])
    parameters["a3"] = a3
    a4 = choice(["relu"])
    parameters["a4"] = a4
    d1 = round(uniform(0.2, 0.5), 1)
    parameters["d1"] = d1
    op = choice(["Adam", "rmsprop", "adamax", "nadam", "sgd"])
    parameters["op"] = op
    ep = randint(50, 100)
    parameters["ep"] = ep
    return parameters
    

## Generate Population

In [10]:
def generate_population(n):
    population = []
    for i in range(n):
        chromosome = initialization()
        population.append(chromosome)
    return population

## Fitness Function ~ In our case we will evaluate accuracy

In [11]:
def fitness_evaluation(model):
    metrics = model.evaluate(X_test, y_test)
    return metrics[1]

## Describe a selection method ~ Roulette wheel selection method

In [12]:
def selection(population_fitness):
  total = sum(population_fitness)
  percentage = [round((x/total) * 100) for x in population_fitness]
  selection_wheel = []
  for pop_index,num in enumerate(percentage):
    selection_wheel.extend([pop_index]*num)
  parent1_ind = choice(selection_wheel)
  parent2_ind = choice(selection_wheel)
  return [parent1_ind, parent2_ind]

## Crossover

In [13]:
def crossover(parent1, parent2):
    child1 = {}
    child2 = {}

    child1["f1"] = choice([parent1["f1"], parent2["f1"]])
    child1["f2"] = choice([parent1["f2"], parent2["f2"]])
    child1["f3"] = choice([parent1["f3"], parent2["f3"]])
    child1["f4"] = choice([parent1["f4"], parent2["f4"]])

    child2["f1"] = choice([parent1["f1"], parent2["f1"]])
    child2["f2"] = choice([parent1["f2"], parent2["f2"]])
    child2["f3"] = choice([parent1["f3"], parent2["f3"]])
    child2["f4"] = choice([parent1["f4"], parent2["f4"]])
    
    child1["k"] = choice([parent1["k"], parent2["k"]])
    child2["k"] = choice([parent1["k"], parent2["k"]])

    child1["a1"] = parent1["a2"]
    child2["a1"] = parent2["a2"]

    child1["a2"] = parent1["a1"]
    child2["a2"] = parent2["a1"]

    child1["a3"] = parent1["a3"]
    child2["a3"] = parent2["a3"]

    child1["a4"] = parent1["a4"]
    child2["a4"] = parent2["a4"]

    child1["d1"] = parent1["d1"]
    child2["d1"] = parent2["d1"]

    child1["op"] = parent1["op"]
    child2["op"] = parent2["op"]

    child1["ep"] = parent1["ep"]
    child2["ep"] = parent2["ep"]

    return [child1, child2]

## Mutation

In [14]:
def mutation(chromosome):
    flag = randint(0,40)
    if flag <= 20:
        chromosome["ep"] += randint(0, 10)
    return chromosome

In [ ]:
generations = 10
threshold = 98
num_pop = 40

population = generate_population(num_pop)

for generation in range(generations):

    population_fitness = []
    for chromosome in population:
        f1 = chromosome["f1"]
        f2 = chromosome["f2"]
        f3 = chromosome["f3"]
        f4 = chromosome["f4"]
        k = chromosome["k"]
        a1 = chromosome["a1"]
        a2 = chromosome["a2"]
        a3 = chromosome["a3"]
        a4 = chromosome["a4"]
        d1 = chromosome["d1"]
        op = chromosome["op"]
        ep = chromosome["ep"]


        try:
            model = build_Classifier(f1, f2, f3, f4, k, a1, a2, a3, a4, d1, op, ep)
            acc = fitness_evaluation(model)
            print("Paramters: ", chromosome)
            print("Accuracy: ", round(acc, 3))            
        except:
            acc = 0
            print("Paramters: ", chromosome)
            print("Invalid paramters - Build fail")
        population_fitness.append(acc)
    parents_ind = selection(population_fitness)
    parents1 = population[parents_ind[0]]
    parents2 = population[parents_ind[1]]

    children = crossover(parent1, parent2)
    child1 = mutation(children[0])
    child2 = mutation(children[1]) 

    population.append(child1)
    population.append(child2)

    print("Generation ", generation+1, " Outcome: ")
    if max(population_fitness) >= threshold:
        print("Obtained desired accuracy: ", max(population_fitness))
        break
    else:
        print("Maximum accuracy in generation {} : {}".format(generation+1, max(population_fitness)))

    first_min = min(population_fitness)
    first_min_ind = population_fitness.index(first_min)
    population.remove(population[first_min_ind])
    second_min = min(population_fitness)
    second_min_ind = population_fitness.index(second_min)
    population.remove(population[second_min_ind])


Paramters:  {'f1': 64, 'f2': 64, 'f3': 128, 'f4': 512, 'k': 5, 'a1': 'relu', 'a2': 'relu', 'a3': 'relu', 'a4': 'relu', 'd1': 0.3, 'op': 'rmsprop', 'ep': 59}
Invalid paramters - Build fail
